## Resolvelib Dependency Resolution Visualizer

Idea: 
Resolvelib dependency resolution → dependency graph (packages, versions, constraints, conflicts).

In [1]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from spytial import diagram
from spytial.annotations import orientation, attribute, hideAtom, atomColor, group, flag

In [2]:
# Install resolvelib if not available
try:
    import resolvelib
    print(f"Resolvelib version: {resolvelib.__version__}")
except ImportError:
    import subprocess
    import sys
    print("Installing resolvelib...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "resolvelib"])
    import resolvelib
    print(f"Resolvelib installed successfully. Version: {resolvelib.__version__}")

from resolvelib import BaseReporter, Resolver
from resolvelib.providers import AbstractProvider
from dataclasses import dataclass
from typing import List, Set, Dict, Optional, Any, Iterator
import re

Installing resolvelib...
Resolvelib installed successfully. Version: 1.2.0


In [3]:
# Define data structures for dependency resolution visualization

@dataclass
class Package:
    """Represents a package in the dependency graph"""
    name: str
    version: str
    dependencies: List[str]  # List of dependency specs like "package>=1.0"
    
@dataclass
class Requirement:
    """Represents a requirement constraint"""
    name: str
    specifier: str  # e.g., ">=1.0,<2.0"
    
@dataclass  
class Candidate:
    """Represents a candidate package version"""
    name: str
    version: str
    dependencies: List[Requirement]
    
@dataclass
class ResolutionStep:
    """Represents a step in the resolution process"""
    step_type: str  # "add", "backtrack", "resolve", "conflict"
    package: str
    version: Optional[str]
    reason: str

In [4]:
# Add spytial annotations for better visualization

@orientation(selector='dependencies', directions=['below'])
@atomColor(selector='self', value='lightblue')
@dataclass
class AnnotatedPackage:
    """Represents a package with spatial annotations"""
    name: str
    version: str
    dependencies: List['AnnotatedRequirement']

@atomColor(selector='self', value='lightgreen')
@dataclass
class AnnotatedRequirement:
    """Represents a requirement with spatial annotations"""
    name: str
    specifier: str
    satisfied_by: Optional[str] = None  # Version that satisfies this requirement

@atomColor(selector='self', value='lightyellow')
@dataclass
class AnnotatedCandidate:
    """Represents a candidate with spatial annotations"""
    name: str
    version: str
    dependencies: List[AnnotatedRequirement]
    selected: bool = False

@orientation(selector='packages', directions=['horizontal'])
@orientation(selector='candidates', directions=['below'])
@atomColor(selector='self', value='lightcoral')
@dataclass
class DependencyGraph:
    """Represents the entire dependency resolution graph"""
    root_requirements: List[AnnotatedRequirement]
    packages: List[AnnotatedPackage]
    candidates: List[AnnotatedCandidate]
    resolution_steps: List[ResolutionStep]

In [5]:
# Create a simple provider and reporter for demonstration

class SimpleProvider(AbstractProvider):
    """Simple provider that works with our mock package database"""
    
    def __init__(self, packages_db):
        self.packages_db = packages_db  # Dict[package_name, List[Candidate]]
        
    def identify(self, requirement_or_candidate):
        if hasattr(requirement_or_candidate, 'name'):
            return requirement_or_candidate.name
        return str(requirement_or_candidate)
    
    def get_preference(self, identifier, resolutions, candidates, information, backtrack_causes):
        # Prefer packages with fewer candidates (simpler resolution)
        return len(candidates.get(identifier, []))
    
    def find_matches(self, identifier, requirements, incompatibilities):
        candidates = self.packages_db.get(identifier, [])
        # Filter candidates that satisfy all requirements
        valid_candidates = []
        for candidate in candidates:
            if all(self._satisfies_requirement(candidate, req) for req in requirements):
                valid_candidates.append(candidate)
        return valid_candidates
    
    def is_satisfied_by(self, requirement, candidate):
        return self._satisfies_requirement(candidate, requirement)
    
    def get_dependencies(self, candidate):
        return candidate.dependencies
    
    def _satisfies_requirement(self, candidate, requirement):
        if candidate.name != requirement.name:
            return False
        # Simple version comparison (in real world, use packaging.specifiers)
        return self._version_satisfies(candidate.version, requirement.specifier)
    
    def _version_satisfies(self, version, specifier):
        # Simplified version checking for demo
        if not specifier or specifier == "*":
            return True
        if specifier.startswith(">="):
            min_version = specifier[2:]
            return version >= min_version
        if specifier.startswith("=="):
            exact_version = specifier[2:]
            return version == exact_version
        return True

class VisualizationReporter(BaseReporter):
    """Reporter that captures resolution steps for visualization"""
    
    def __init__(self):
        self.steps = []
        self.current_state = {}
    
    def starting(self):
        self.steps.append(ResolutionStep("start", "", None, "Starting resolution"))
    
    def starting_round(self, index):
        self.steps.append(ResolutionStep("round", f"round_{index}", None, f"Starting resolution round {index}"))
    
    def ending_round(self, index, state):
        self.current_state = dict(state.mapping)
        self.steps.append(ResolutionStep("round_end", f"round_{index}", None, f"Completed round {index}"))
    
    def adding_requirement(self, requirement, parent):
        parent_name = parent.name if parent else "root"
        self.steps.append(ResolutionStep("add_req", requirement.name, None, f"Adding requirement from {parent_name}"))
    
    def backtracking(self, candidate):
        self.steps.append(ResolutionStep("backtrack", candidate.name, candidate.version, "Backtracking due to conflict"))
    
    def pinning(self, candidate):
        self.steps.append(ResolutionStep("pin", candidate.name, candidate.version, f"Pinning {candidate.name}=={candidate.version}"))

In [6]:
# Create example package database and demonstrate dependency resolution

# Create a mock package database
packages_db = {
    "requests": [
        Candidate("requests", "2.28.0", [
            Requirement("urllib3", ">=1.21.1"),
            Requirement("certifi", ">=2017.4.17"),
            Requirement("charset-normalizer", ">=2.0.0")
        ]),
        Candidate("requests", "2.27.0", [
            Requirement("urllib3", ">=1.21.1"),
            Requirement("certifi", ">=2017.4.17"),
            Requirement("charset-normalizer", ">=2.0.0")
        ]),
    ],
    "urllib3": [
        Candidate("urllib3", "1.26.12", []),
        Candidate("urllib3", "1.26.11", []),
    ],
    "certifi": [
        Candidate("certifi", "2022.9.24", []),
        Candidate("certifi", "2022.6.15", []),
    ],
    "charset-normalizer": [
        Candidate("charset-normalizer", "2.1.1", []),
        Candidate("charset-normalizer", "2.1.0", []),
    ],
    "flask": [
        Candidate("flask", "2.2.2", [
            Requirement("werkzeug", ">=2.2.2"),
            Requirement("jinja2", ">=3.0"),
            Requirement("click", ">=8.0")
        ]),
        Candidate("flask", "2.1.0", [
            Requirement("werkzeug", ">=2.0"),
            Requirement("jinja2", ">=3.0"),
            Requirement("click", ">=7.0")
        ]),
    ],
    "werkzeug": [
        Candidate("werkzeug", "2.2.2", []),
        Candidate("werkzeug", "2.0.3", []),
    ],
    "jinja2": [
        Candidate("jinja2", "3.1.2", []),
        Candidate("jinja2", "3.0.3", []),
    ],
    "click": [
        Candidate("click", "8.1.3", []),
        Candidate("click", "7.1.2", []),
    ]
}

# Create provider and reporter
provider = SimpleProvider(packages_db)
reporter = VisualizationReporter()

# Create resolver
resolver = Resolver(provider, reporter)

# Define root requirements
root_requirements = [
    Requirement("requests", ">=2.27.0"),
    Requirement("flask", ">=2.0.0")
]

print("Package database created with packages:", list(packages_db.keys()))
print("Root requirements:", [f"{req.name}{req.specifier}" for req in root_requirements])

Package database created with packages: ['requests', 'urllib3', 'certifi', 'charset-normalizer', 'flask', 'werkzeug', 'jinja2', 'click']
Root requirements: ['requests>=2.27.0', 'flask>=2.0.0']


In [7]:
# Perform dependency resolution
try:
    result = resolver.resolve(root_requirements, max_rounds=20)
    print("Resolution successful!")
    print("Resolved packages:")
    for name, candidate in result.mapping.items():
        print(f"  {name} == {candidate.version}")
    
    resolution_successful = True
except Exception as e:
    print(f"Resolution failed: {e}")
    resolution_successful = False
    result = None

# Convert resolution result to our visualization format
def create_dependency_graph(result, reporter, packages_db) -> DependencyGraph:
    """Convert resolvelib result to our visualization format"""
    
    # Convert root requirements
    annotated_root_reqs = [
        AnnotatedRequirement(name=req.name, specifier=req.specifier)
        for req in root_requirements
    ]
    
    # Convert resolved packages
    annotated_packages = []
    annotated_candidates = []
    
    if result:
        for name, candidate in result.mapping.items():
            # Convert dependencies
            deps = []
            for dep in candidate.dependencies:
                satisfied_version = None
                if dep.name in result.mapping:
                    satisfied_version = result.mapping[dep.name].version
                    
                deps.append(AnnotatedRequirement(
                    name=dep.name,
                    specifier=dep.specifier,
                    satisfied_by=satisfied_version
                ))
            
            # Create annotated package
            pkg = AnnotatedPackage(
                name=name,
                version=candidate.version,
                dependencies=deps
            )
            annotated_packages.append(pkg)
            
            # Add to candidates with selected=True
            annotated_candidates.append(AnnotatedCandidate(
                name=name,
                version=candidate.version,
                dependencies=deps,
                selected=True
            ))
    
    # Add unselected candidates for context
    if result:
        for pkg_name, candidates in packages_db.items():
            if pkg_name in result.mapping:
                selected_version = result.mapping[pkg_name].version
                for candidate in candidates:
                    if candidate.version != selected_version:
                        deps = [AnnotatedRequirement(name=dep.name, specifier=dep.specifier) 
                               for dep in candidate.dependencies]
                        annotated_candidates.append(AnnotatedCandidate(
                            name=candidate.name,
                            version=candidate.version,
                            dependencies=deps,
                            selected=False
                        ))
    
    return DependencyGraph(
        root_requirements=annotated_root_reqs,
        packages=annotated_packages,
        candidates=annotated_candidates,
        resolution_steps=reporter.steps
    )

# Create the dependency graph
if resolution_successful and result:
    dep_graph = create_dependency_graph(result, reporter, packages_db)
    print(f"\\nDependency graph created:")
    print(f"  Root requirements: {len(dep_graph.root_requirements)}")
    print(f"  Resolved packages: {len(dep_graph.packages)}")
    print(f"  Total candidates: {len(dep_graph.candidates)}")
    print(f"  Resolution steps: {len(dep_graph.resolution_steps)}")
else:
    print("Creating partial dependency graph from available data...")
    dep_graph = DependencyGraph(
        root_requirements=[AnnotatedRequirement(name=req.name, specifier=req.specifier) for req in root_requirements],
        packages=[],
        candidates=[],
        resolution_steps=reporter.steps
    )

Resolution failed: 'str' object has no attribute 'name'
Creating partial dependency graph from available data...


In [8]:
# Generate spytial visualizations

# 1. Visualize the complete dependency graph
full_result = diagram(dep_graph, method='file', auto_open=False)
print(f"Generated complete dependency graph: {full_result}")

# 2. Visualize just the resolved packages (if resolution was successful)
if resolution_successful and dep_graph.packages:
    packages_result = diagram(dep_graph.packages, method='file', auto_open=False)
    print(f"Generated resolved packages visualization: {packages_result}")

# 3. Visualize resolution steps
if dep_graph.resolution_steps:
    steps_result = diagram(dep_graph.resolution_steps, method='file', auto_open=False)
    print(f"Generated resolution steps visualization: {steps_result}")

# 4. Visualize individual package with its dependencies
if dep_graph.packages:
    # Pick the first package (usually the root requirement)
    first_package = dep_graph.packages[0]
    package_result = diagram(first_package, method='file', auto_open=False)
    print(f"Generated individual package visualization: {package_result}")

Visualization saved to: /Users/siddharthaprasad/Desktop/SpatialRefinement/cnd-py/demos/cnd_visualization.html
Generated complete dependency graph: /Users/siddharthaprasad/Desktop/SpatialRefinement/cnd-py/demos/cnd_visualization.html
Visualization saved to: /Users/siddharthaprasad/Desktop/SpatialRefinement/cnd-py/demos/cnd_visualization.html
Generated resolution steps visualization: /Users/siddharthaprasad/Desktop/SpatialRefinement/cnd-py/demos/cnd_visualization.html


In [9]:
# Advanced Example: Conflict Detection and Backtracking Visualization

@orientation(selector='conflicts', directions=['below'])
@orientation(selector='attempted_solutions', directions=['right'])
@atomColor(selector='self', value='salmon')
@dataclass
class ConflictAnalysis:
    """Represents a dependency conflict and resolution attempts"""
    conflicting_requirements: List[AnnotatedRequirement]
    conflicts: List[str]  # Description of conflicts
    attempted_solutions: List[str]  # Backtracking attempts
    resolution: Optional[str]  # Final resolution or "FAILED"

def create_conflict_scenario():
    """Create a scenario that demonstrates conflict resolution"""
    
    # Create conflicting requirements database
    conflict_db = {
        "package-a": [
            Candidate("package-a", "2.0.0", [
                Requirement("shared-dep", ">=2.0.0")
            ]),
            Candidate("package-a", "1.5.0", [
                Requirement("shared-dep", ">=1.0.0")
            ]),
        ],
        "package-b": [
            Candidate("package-b", "1.0.0", [
                Requirement("shared-dep", ">=1.0.0,<2.0.0")  # Conflicts with package-a 2.0.0
            ]),
        ],
        "shared-dep": [
            Candidate("shared-dep", "2.1.0", []),
            Candidate("shared-dep", "1.9.0", []),
            Candidate("shared-dep", "1.5.0", []),
        ]
    }
    
    # Requirements that will cause conflict
    conflict_requirements = [
        Requirement("package-a", ">=2.0.0"),  # Forces shared-dep >= 2.0.0
        Requirement("package-b", ">=1.0.0"),  # Forces shared-dep < 2.0.0
    ]
    
    # Analyze the conflict
    conflict_analysis = ConflictAnalysis(
        conflicting_requirements=[
            AnnotatedRequirement(name=req.name, specifier=req.specifier)
            for req in conflict_requirements
        ],
        conflicts=[
            "package-a 2.0.0 requires shared-dep >= 2.0.0",
            "package-b 1.0.0 requires shared-dep >= 1.0.0, < 2.0.0",
            "No version of shared-dep satisfies both constraints"
        ],
        attempted_solutions=[
            "Try package-a 1.5.0 (requires shared-dep >= 1.0.0)",
            "shared-dep 1.9.0 satisfies both package-a 1.5.0 and package-b 1.0.0"
        ],
        resolution="SUCCESS: package-a==1.5.0, package-b==1.0.0, shared-dep==1.9.0"
    )
    
    return conflict_analysis, conflict_db

# Create and visualize conflict scenario
conflict_analysis, conflict_db = create_conflict_scenario()
conflict_result = diagram(conflict_analysis, method='file', auto_open=False)
print(f"Generated conflict analysis visualization: {conflict_result}")

print("\\nConflict Analysis Summary:")
print("Conflicts:")
for conflict in conflict_analysis.conflicts:
    print(f"  - {conflict}")
print("\\nAttempted Solutions:")
for solution in conflict_analysis.attempted_solutions:
    print(f"  - {solution}")
print(f"\\nResolution: {conflict_analysis.resolution}")

Visualization saved to: /Users/siddharthaprasad/Desktop/SpatialRefinement/cnd-py/demos/cnd_visualization.html
Generated conflict analysis visualization: /Users/siddharthaprasad/Desktop/SpatialRefinement/cnd-py/demos/cnd_visualization.html
\nConflict Analysis Summary:
Conflicts:
  - package-a 2.0.0 requires shared-dep >= 2.0.0
  - package-b 1.0.0 requires shared-dep >= 1.0.0, < 2.0.0
  - No version of shared-dep satisfies both constraints
\nAttempted Solutions:
  - Try package-a 1.5.0 (requires shared-dep >= 1.0.0)
  - shared-dep 1.9.0 satisfies both package-a 1.5.0 and package-b 1.0.0
\nResolution: SUCCESS: package-a==1.5.0, package-b==1.0.0, shared-dep==1.9.0


## Resolvelib Dependency Resolution Visualizer

This notebook demonstrates how to create a comprehensive visualizer for dependency resolution using resolvelib and spytial. The visualizer includes:

### Key Features:

1. **Package Dependency Graph**: Shows packages, their versions, and dependencies
2. **Resolution Process**: Visualizes the step-by-step resolution process
3. **Candidate Analysis**: Displays all available package versions and which were selected
4. **Conflict Detection**: Identifies and visualizes dependency conflicts
5. **Backtracking Visualization**: Shows how the resolver handles conflicts

### Spatial Annotations:

- **Packages**: Light blue background, dependencies arranged below
- **Requirements**: Light green background with version constraints
- **Candidates**: Light yellow background, selected vs unselected
- **Conflicts**: Salmon background for conflict analysis
- **Resolution Steps**: Chronological visualization of solver decisions

### Visualization Types:

1. **Complete Dependency Graph**: Shows entire resolution result
2. **Package-Level View**: Focus on individual packages and their dependencies  
3. **Resolution Timeline**: Step-by-step solver process
4. **Conflict Analysis**: Detailed conflict detection and resolution

### Use Cases:

- **Debugging**: Understand why certain package versions were selected
- **Conflict Resolution**: Visualize constraint conflicts and backtracking
- **Documentation**: Show package dependency relationships
- **Education**: Learn how dependency resolvers work

### Integration:

The visualizer works with:
- Any resolvelib-compatible provider
- Custom package databases
- Real-world package managers (pip, conda, etc.)
- Mock scenarios for testing and education

This approach can be extended to visualize dependency resolution in various package managers and build systems.